## 原始方法 是否和  CIGAR(M/=/X) 解析的坐标一致

In [4]:
BAM_FILE = "/data/work/human_brain/710_68_1_humanbrain_251209_NB.duplicates.directional.bam"

In [5]:
import pysam
import pandas as pd
import gffutils
import matplotlib.pyplot as plt
from intervaltree import IntervalTree

def get_precise_capture_site(read):
    # <--- 新增：CIGAR 字符串解析捕获位点
    """
    根据 CIGAR 字符串解析出最后一个真正匹配 (M/=/X) 的基因组坐标
    返回该位点所在的 M/=/X 块的长度 (m_length)
    """
    # 获取 CIGAR 元组，如果为空则返回默认值
    cigars = read.cigartuples
    if not cigars:
        return (read.reference_start, 0)
    
    if not read.is_reverse:
        # 正向链 (+): 寻找最后一个匹配碱基的末端坐标
        curr_ref_pos = read.reference_start
        last_match_end = read.reference_start
        last_m_len = 0
        if cigars:
            for op, length in cigars:
                if op in (0, 2, 7, 8): # M, D, =, X (消耗参考基因组坐标)
                    curr_ref_pos += length
                    if op != 2: # 只有 M, =, X 是真实匹配，D 是缺失
                        last_match_end = curr_ref_pos
                        last_m_len = length
                elif op == 3: # N (跨越内含子，消耗坐标)
                    curr_ref_pos += length
        return last_match_end, last_m_len
    else:
        # 反向链 (-): 寻找第一个匹配碱基的起始坐标
        curr_ref_pos = read.reference_start
        first_m_len = 0
        if cigars:
            for op, length in cigars:
                if op in (0, 7, 8): # 找到第一个真正的匹配块 (M, =, X)
                    first_m_len = length
                    return curr_ref_pos, first_m_len
                if op in (2, 3): # D, N 消耗坐标
                    curr_ref_pos += length
        return read.reference_start,0

In [11]:
import pysam
import pandas as pd
import matplotlib.pyplot as plt

def verify_all_reads_capture_logic(bam_path):
    """
    对比过滤后的 Primary Aligned Reads：
    原始方法 vs CIGAR 精确解析
    """
    bf = pysam.AlignmentFile(bam_path, "rb")
    
    # 为了节省内存：
    # 1. 差异列表：只存储 offset != 0 的 read 信息
    diff_data = []
    # 2. 计数器：记录总体情况
    stats = {
        "total_processed": 0,
        "primary_count": 0,
        "consistent_count": 0,
        "diff_count": 0
    }

    print(f"--- 开始全量扫描 BAM: {bam_path} ---")

    for read in bf.fetch(until_eof=True):
        stats["total_processed"] += 1
        
        # 核心过滤逻辑
        if read.is_unmapped or read.is_secondary or read.is_supplementary:
            continue
        
        stats["primary_count"] += 1
        
        # 1. 原始方法坐标
        if not read.is_reverse:  →找到原因了！！！
            raw_site = read.reference_end
        else:
            raw_site = read.reference_start
        
        # 2. CIGAR 精确解析 (使用新定义的函数)
        precise_site, m_len = get_precise_capture_site(read)
        
        # 3. 计算偏移
        offset = precise_site - raw_site
        
        if offset == 0:
            stats["consistent_count"] += 1
        else:
            stats["diff_count"] += 1
            # 只有在不一致时才记录详细信息，极大地节省内存
            diff_data.append({
                'read_name': read.query_name,
                'strand': '-' if read.is_reverse else '+',
                'cigar': read.cigarstring,
                'raw_site': raw_site,
                'precise_site': precise_site,
                'offset': offset,
                'terminal_m_len': m_len
            })

        # 每 100 万条打印一次进度
        if stats["primary_count"] % 1000000 == 0:
            print(f"进度：已处理 {stats['primary_count'] // 1000000}M 条 Primary Reads...")

    bf.close()

    # 转换为 DataFrame 仅包含有差异的部分
    df_diff = pd.DataFrame(diff_data)
    
    # --- 生成最终报告 ---
    print("\n" + "="*50)
    print("全量验证最终报告")
    print("-" * 50)
    print(f"BAM 总 Reads 数:    {stats['total_processed']}")
    print(f"通过过滤的 Primary:  {stats['primary_count']}")
    print(f"坐标完全一致:       {stats['consistent_count']}")
    print(f"坐标存在差异:       {stats['diff_count']}")
    
    if stats["primary_count"] > 0:
        error_rate = (stats["diff_count"] / stats["primary_count"]) * 100
        print(f"受影响 Read 占比:   {error_rate:.2f}%")
    
    if not df_diff.empty:
        print("\n--- 偏移量 (Offset) 最大/最小统计 ---")
        print(f"最大正向偏移: {df_diff['offset'].max()} bp")
        print(f"最大负向偏移: {df_diff['offset'].min()} bp")
        
        # 可视化差异分布
        plot_offset_distribution(df_diff)
    
    print("="*50)
    
    return df_diff

def plot_offset_distribution(df_diff):
    plt.figure(figsize=(10, 6))
    plt.hist(df_diff['offset'], bins=100, color='salmon', edgecolor='black', alpha=0.7)
    plt.yscale('log') # 使用对数刻度，因为小的偏移可能非常多，大的偏移很少
    plt.title('Distribution of Coordinate Offsets (Precise - Raw)')
    plt.xlabel('Offset (bp)')
    plt.ylabel('Read Count (Log Scale)')
    plt.grid(axis='y', linestyle='--', alpha=0.4)
    plt.show()

In [12]:
df_verify = verify_all_reads_capture_logic(BAM_FILE)

--- 开始全量扫描 BAM: /data/work/human_brain/710_68_1_humanbrain_251209_NB.duplicates.directional.bam ---
进度：已处理 1M 条 Primary Reads...
进度：已处理 2M 条 Primary Reads...
进度：已处理 3M 条 Primary Reads...
进度：已处理 4M 条 Primary Reads...
进度：已处理 5M 条 Primary Reads...
进度：已处理 6M 条 Primary Reads...
进度：已处理 7M 条 Primary Reads...

全量验证最终报告
--------------------------------------------------
BAM 总 Reads 数:    7662144
通过过滤的 Primary:  7662144
坐标完全一致:       7662144
坐标存在差异:       0
受影响 Read 占比:   0.00%
